In [1]:
import os

In [2]:
from google.cloud import bigquery
bq = bigquery.Client(project="cs544-spring2023")
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [3]:
%load_ext google.cloud.bigquery

In [4]:
# %%bigquery
# SELECT *
# FROM bigquery-public-data.geo_us_boundaries.counties
# LIMIT 5

In [5]:
#Part 1: County Data (Public Dataset)

In [6]:
#q1 what is the geo_id for Dane county? (note that Madison is in Dane county).
q = bq.query("""
SELECT geo_id, county_name
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE (state_fips_code = '55') AND (county_name LIKE 'Dane') --55 is WI
""", job_config=no_cache)
q.to_dataframe().loc[0][0]

'55025'

In [7]:
#Q2 how many counties are there per state?
p = bq.query("""
SELECT state_fips_code, COUNT(county_fips_code) AS count
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY count DESC
LIMIT 5
""", job_config=no_cache)
p.to_dataframe().set_index("state_fips_code").to_dict()["count"]

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [8]:
#q3 about how much should the queries for the last two questions cost?
q3 = {}
q3['q1'] = ((q.total_bytes_billed / 1024**4) * 500)
q3['q2'] = ((p.total_bytes_billed / 1024**4) * 500)
q3

{'q1': 0.00476837158203125, 'q2': 0.00476837158203125}

In [9]:
#Part 2: HDMA Data (Parquet in GCS)

In [10]:
if not os.path.exists("hdma-wi-2021.parquet"):
    !wget https://pages.cs.wisc.edu/~harter/cs639/data/hdma-wi-2021.parquet
else:
    print("hdma-wi-2021.parquet already downloaded")

hdma-wi-2021.parquet already downloaded


In [11]:
# project.dataset.table
dataset = bigquery.Dataset("cs544-spring2023.p7")
bq.create_dataset(dataset, exists_ok=True)

Dataset(DatasetReference('cs544-spring2023', 'p7'))

In [12]:
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
job = bq.load_table_from_uri(["gs://may4/hdma-wi-2021.parquet"],
                             "cs544-spring2023.p7.hdma", job_config=config)
job.result()

LoadJob<project=cs544-spring2023, location=US, id=7da8e8d0-c231-4c48-81cf-6b1488e6f65e>

In [13]:
#q4  what are the datasets in your GCP project?
[ds.dataset_id for ds in bq.list_datasets("cs544-spring2023")] # PASTE project name

['p7']

In [14]:
#q5 how many loan applications are there in the HDMA data for each county?
q = bq.query("""
SELECT L2.county_name, COUNT(L1) AS count
FROM cs544-spring2023.p7.hdma AS L1
    LEFT JOIN bigquery-public-data.geo_us_boundaries.counties AS L2
    ON L1.county_code = L2.county_fips_code
WHERE L1.county_code != 'None'
GROUP BY L2.county_name
ORDER BY count DESC
LIMIT 10
""")
q.to_dataframe().set_index("county_name").to_dict()["count"]

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [15]:
#Part 3: Application Data (Google Sheet Linked to Form)

In [16]:
url = "https://drive.google.com/open?id=1e2qLPyxZ7s5ibMyEg7bxX2wWYAQD2ROAUcZuv8fqhnA"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = ["https://docs.google.com/spreadsheets/d/1e2qLPyxZ7s5ibMyEg7bxX2wWYAQD2ROAUcZuv8fqhnA/edit?pli=1#gid=2129344515"]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(dataset.table("applications"))
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [17]:
#q6 how many applications are there with your chosen income?
q = bq.query("""
SELECT COUNT(*) count
FROM cs544-spring2023.p7.applications
WHERE income = 80000
ORDER BY count DESC
""")
q.to_dataframe().loc[0][0]

5

In [18]:
%%bigquery
CREATE OR REPLACE TABLE cs544-spring2023.p7.houses

AS

SELECT *, ST_GEOGPOINT(longitude, latitude) AS loc
FROM cs544-spring2023.p7.applications

Query is running:   0%|          |

""


In [19]:
#q7 how many applications are there in the Google sheet per WI county?
q = bq.query("""
SELECT L2.county_name, COUNT(L1) AS count, 
FROM cs544-spring2023.p7.houses as L1
    CROSS JOIN bigquery-public-data.geo_us_boundaries.counties as L2
WHERE ST_WITHIN(L1.loc,L2.county_geom) AND (L2.state_fips_code = '55')
GROUP BY county_name
ORDER BY count DESC
""")
q.to_dataframe().set_index("county_name").to_dict()["count"]

{'Dane': 30,
 'Door': 5,
 'Jefferson': 4,
 'Brown': 3,
 'Bayfield': 3,
 'Milwaukee': 2,
 'Columbia': 2,
 'Walworth': 2,
 'Sauk': 1,
 'Ashland': 1,
 'Green Lake': 1,
 'La Crosse': 1,
 'Barron': 1,
 'Iowa': 1,
 'Kewaunee': 1,
 'Waukesha': 1,
 'Adams': 1,
 'Oneida': 1,
 'Sheboygan': 1,
 'Iron': 1,
 'Monroe': 1}

In [20]:
#Part 4: Machine Learning

In [21]:
%%bigquery
CREATE OR REPLACE MODEL `cs544-spring2023.p7.m1`
OPTIONS(model_type='LINEAR_REG', INPUT_LABEL_COLS=['loan_amount'])

AS

SELECT income, loan_term, loan_amount
FROM `cs544-spring2023.p7.hdma`

Query is running:   0%|          |

""


In [22]:
%%bigquery df
SELECT * 
FROM
ML.EVALUATE(
    MODEL `cs544-spring2023.p7.m1`,
    (
        SELECT income, loan_term, loan_amount
        FROM `cs544-spring2023.p7.hdma`
    )
)


Query is running:   0%|          |

Downloading:   0%|          |

In [23]:
#q8 what is your model's r2_score on the HDMA dataset on which it was trained?
df.iloc[0].at["r2_score"]

0.29165412288020864

In [24]:
%%bigquery df
SELECT * 
FROM
ML.WEIGHTS (MODEL `cs544-spring2023.p7.m1`)

Query is running:   0%|          |

Downloading:   0%|          |

In [25]:
#q9 what is the coefficient weight on the income column?
df[df["processed_input"] == "income"]["weight"][0]

0.8057773037176441

In [26]:
#q10: what ratio of the loan applications in the Google form are for amounts greater than the model 
#would predict, given income?

q = bq.query("""
SELECT SUM(Bigger) / COUNT(*) AS Ratio
FROM(
SELECT loan_amount, predicted_loan_amount, IF(loan_amount > predicted_loan_amount, 1, 0) AS Bigger
FROM ML.PREDICT(
    MODEL `cs544-spring2023.p7.m1`,
    (
        SELECT income, 360 AS loan_term, loan_amount
        FROM `cs544-spring2023.p7.hdma`
    )
  )
)
""")
q.to_dataframe()["Ratio"].loc[0]

0.3593285349802225